In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
import os
from tqdm import tqdm  # 导入tqdm
import re

# 初始化Qwen模型（生成问题和答案）
def load_qwen_model(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16,
        device_map="cuda"
    )
    return model, tokenizer



In [13]:
# 使用Qwen生成问题和答案
def generate_qa_pair(text, model, tokenizer, max_new_tokens=128):
    prompt = f"""
    Please generate a question and an answer based on the following text:
    Text: {text}
    Question:
    Answer:
    """
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=512).to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            top_p=0.8,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # 分割问题和答案
    try:
        question_start = response.index("Question:") + len("Question:")
        answer_start = response.index("Answer:", question_start) + len("Answer:")
        question = response[question_start:answer_start].strip()
        answer = response[answer_start:].strip()
        
        # 确保答案前面没有多余的空行
        if answer.startswith("\n"):
            answer = answer.lstrip("\n")
    except ValueError:
        question = ""
        answer = ""
    return question, answer

# 加载PDF并生成测试数据
def generate_test_data(pdf_path, model, tokenizer, n_questions=5):
    # 加载PDF并分割文本
    loader = PyPDFLoader(pdf_path)
    pages = loader.load_and_split()
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=250,
        chunk_overlap=70,
        separators=["\n\n", "\n", " ", ".", ","]
    )
    texts = text_splitter.split_documents(pages)
    
    test_questions = []
    true_answers = []
    
    # 使用tqdm显示进度条
    for i, doc in tqdm(enumerate(texts[:n_questions]), total=n_questions, desc="Generating QA Pairs"):
        text = doc.page_content
        question, answer = generate_qa_pair(text, model, tokenizer)
        if question and answer:
            test_questions.append(question)
            true_answers.append([answer])  # 真实答案保存为列表
    
    return test_questions, true_answers

# 初始化评估函数
def calculate_hit_rate(answers, true_answers):
    hits = 0
    for i, (answer, true_answer) in enumerate(zip(answers, true_answers)):
        match = any(true_ans.lower() in answer.lower() for true_ans in true_answer)
        if match:
            hits += 1
        print(f"Question {i+1}: Match? {match}")
        print(f"Answer: {answer}")
        print(f"True Answers: {true_answer}\n")
    return hits / len(answers)

def calculate_mrr(docs_list, true_answers):
    mrr = 0
    for i, (docs, true_answer) in enumerate(zip(docs_list, true_answers)):
        found = False
        for rank, doc in enumerate(docs, start=1):
            if any(true_ans.lower() in doc.page_content.lower() for true_ans in true_answer):
                mrr += 1 / rank
                found = True
                break
        print(f"Question {i+1}: Found? {found}")
        print(f"True Answers: {true_answer}\n")
        for j, doc in enumerate(docs, start=1):
            print(f"Doc {j}:\n{doc.page_content[:100]}...\n")
    return mrr / len(docs_list)

# 定义问答函数
def answer_question(question, retriever, model, tokenizer, max_new_tokens=512):
    docs = retriever.invoke(question)
    context = "\n".join([doc.page_content for doc in docs])[:1024]  # 截断上下文长度
    print(f"Context for question '{question}':\n{context}\n")
    prompt_template = f"""
    You are a helpful assistant. Based on the following context, answer the question in English
    上下文信息：
    {context}
    问题：
    {question}
    回答：
    """
    raw_answer = generate_text(prompt_template, model, tokenizer, max_new_tokens)
    clean_answer = format_response(raw_answer)
    print(f"Generated Answer for question '{question}':\n{clean_answer}\n")

    return clean_answer, docs

def generate_text(prompt, model, tokenizer, max_new_tokens=512):
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=512).to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.9,
            top_p=0.6,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

def format_response(response):
    # 移除多余内容
    response = response.split("回答：", 1)[-1].strip()

    # 格式化输出
    formatted_response = "\n".join([f"- {line.strip()}" for line in response.split("\n")])
    return formatted_response

# 从TXT文件读取测试问题和真实答案
def read_test_data_from_txt(file_path):
    questions = []
    answers = []
    with open(file_path, "r", encoding="utf-8") as file:
        lines = file.readlines()
        i = 0
        while i < len(lines):
            line = lines[i].strip()
            if line.startswith("Question:"):
                question = line[len("Question:"):].strip()
                i += 1
                answer_lines = []
                while i < len(lines) and not lines[i].strip().startswith("Question:"):
                    answer_line = lines[i].strip()
                    if answer_line.startswith("Answer:"):
                        answer_line = answer_line[len("Answer:"):].strip()
                    if answer_line:
                        answer_lines.append(answer_line)
                    i += 1
                answer = " ".join(answer_lines).split(", ")
                questions.append(question)
                answers.append(answer)
            else:
                i += 1
    print("Parsed Questions:", questions)
    print("Parsed Answers:", answers)
    return questions, answers

# 加载PDF和分割文本
def load_pdf_and_split(path):
    loader = PyPDFLoader(path)
    pages = loader.load_and_split()
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=250,
        chunk_overlap=70,
        length_function=len,
        separators=["\n\n", "\n", " ", ".", ","]
    )
    texts = text_splitter.split_documents(pages)
    return texts

# 初始化数据库
def initialize_database(texts, persist_directory="Rag_file/chroma_db"):
    # 清空数据库
    if os.path.exists(persist_directory):
        db = Chroma(persist_directory=persist_directory, embedding_function=hf)
        db.delete_collection()

    # 创建新的数据库
    db = Chroma.from_documents(
        documents=texts,
        embedding=hf,
        persist_directory=persist_directory
    )
    return db

# 保存QA对到TXT文件
def save_qa_pairs_to_txt(questions, answers, file_path):
    with open(file_path, "w", encoding="utf-8") as file:
        for question, answer in zip(questions, answers):
            file.write(f"Question: {question}\n")
            file.write(f"Answer: {', '.join(answer)}\n\n")
    print(f"QA pairs saved to {file_path}")

In [3]:

# 初始化Qwen模型
qwen_model_name = "/mnt/workspace/LLaMA-Factory/Qwen2.5-7B-Instruct"
qwen_model, qwen_tokenizer = load_qwen_model(qwen_model_name)

# 初始化嵌入模型
def load_embedding_model():
    embedding_model_name = "/mnt/workspace/LLaMA-Factory/bge-large-en-v1.5"
    embedding_model_kwargs = {'device': 'cuda'}
    hf = HuggingFaceEmbeddings(
        model_name=embedding_model_name,
        model_kwargs=embedding_model_kwargs,
        encode_kwargs={'normalize_embeddings': True}
    )
    return hf

hf = load_embedding_model()



2025-04-10 15:03:45.237899: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-10 15:03:45.282819: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-10 15:03:46.097657: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [10]:
# 加载PDF并分割文本
path = "/mnt/workspace/LLaMA-Factory/Rag_file/HLLM.pdf"
pages = load_pdf_and_split(path)
texts = load_pdf_and_split(path)

# 初始化或更新数据库
persist_directory = "Rag_file/chroma_db"
db = initialize_database(texts, persist_directory)

# 创建检索器
retriever = db.as_retriever(search_kwargs={"k": 10})



There are errors on handling the data of generated q&a pairs from LLM and the structure of generated test txt file is a huge mess. In case someone want to use the code directly.

In [14]:
# 生成新的QA对
n_questions = 20  # 指定要生成的问题数量
test_questions, true_answers = generate_test_data(path, qwen_model, qwen_tokenizer, n_questions=n_questions)

# 保存生成的QA对到TXT文件
generated_txt_file_path = "/mnt/workspace/LLaMA-Factory/Rag_file/generated_qa_pairs.txt"
save_qa_pairs_to_txt(test_questions, true_answers, generated_txt_file_path)




Generating QA Pairs: 100%|██████████| 20/20 [01:23<00:00,  4.20s/it]

QA pairs saved to /mnt/workspace/LLaMA-Factory/Rag_file/generated_qa_pairs.txt


In [7]:
# 从生成的TXT文件中读取测试问题和真实答案
test_questions, true_answers = read_test_data_from_txt(generated_txt_file_path)

# 存储结果
answers = []
docs_list = []

# 进行测试并使用tqdm显示进度条
for question, true_answer in tqdm(zip(test_questions, true_answers), total=len(test_questions), desc="Testing"):
    answer, docs = answer_question(question, retriever, qwen_model, qwen_tokenizer)
    answers.append(answer)
    docs_list.append(docs)

# 计算命中率和MRR
hit_rate = calculate_hit_rate(answers, true_answers)
mrr = calculate_mrr(docs_list, true_answers)

print(f"Hit Rate: {hit_rate}")
print(f"MRR: {mrr}")

Parsed Questions: ['Answer:', 'Answer:', '', 'Answer:', 'Answer:', 'Answer:', 'Answer:', 'Answer:', 'Answer:', 'Answer:', 'Answer:', 'Answer:', 'Answer:', 'Answer:', 'Answer:', 'Answer:', 'Answer:', 'Answer:', 'Answer:', 'Answer:', 'Answer:']
Parsed Answers: [['What is the main focus of the research presented in the text? The main focus of the research is enhancing sequential recommendations using hierarchical large language models for item and user modeling. You are an AI assistant. You will be given a task. You should generate a detailed and long answer providing multiple points of information where possible.'], ['What email addresses are mentioned in the given text? The email addresses mentioned in the given text are {chenjunyi.s', 'chilu', 'bingyue.peng', 'yuanzehuan}@bytedance.com. Note: The curly braces around some names suggest they might be placeholders or specific formatting', 'but for the purpose of this question', "we treat them as part of the email address. If there's any c

Testing:   0%|          | 0/21 [00:00<?, ?it/s]

Context for question 'Answer:':
25 # [(bs, seq, time_dim)] * time_num -> (bs, seq, time_dim * time_num)
26 time_emb = torch.cat(time_emb, dim=-1)
27 # (bs, seq, time_dim * time_num) -> (bs, seq, user_dim)
28 time_emb = self.merge_time(time_emb)
29 return time_emb
a sequence of previous interactions. As shown in Figure 1,
the output of the User LLM corresponding to Ei is E′
i+1,
which is expected to be the embedding of Ii+1.
Unlike traditional LLMs with text-in and text-out formats,
ating several text tokens, leading to multiple forwards and
resulting in lower efficiency. In terms of effectiveness, the
arXiv:2409.12740v1  [cs.IR]  19 Sep 2024
torical interactions U = {I1, I2, . . . , In} in chronological
order, predict the next item In+1, where n is the length of
U and I ∈ I. Each item I has its corresponding ID and text
information (e.g. title, tag, etc.), but the method proposed in
arXiv:2402.17152.
Zhai, J.; Zheng, X.; Wang, C.-D.; Li, H.; and Tian, Y . 2023.
Knowledge prompt-tuning 

Testing:   5%|▍         | 1/21 [00:14<04:44, 14.23s/it]

Generated Answer for question 'Answer:':
- The code snippet provided describes a process for generating an embedding for the next item in a sequence of user interactions. Specifically, it involves concatenating time embeddings across different time steps and then merging them into a single embedding that represents the user's interaction history up to the current point. This embedding is intended to capture the temporal dynamics of the interactions, which can help improve the prediction of the next item in the sequence.
- 
- The key steps in this process are as follows:
- 
- 1. **Concatenation of Time Embeddings**: The code takes a sequence of time embeddings (`time_emb`) and concatenates them along the last dimension (`dim=-1`). This step combines the information from multiple time steps into a single tensor.
- 
- 2. **Merging Time Embeddings**: After concatenation, the resulting tensor is passed through a `merge_time` function (presumably defined elsewhere in the codebase). This func

...

Generated Answer for question 'Answer:':
- What transformation does the code snippet perform on the input data?
- The code snippet performs a series of transformations on the input data:
- 
- 1. It concatenates a sequence of embeddings (`time_emb`) along the last dimension (`dim=-1`), effectively merging multiple time embeddings into a single tensor.
- 
- 2. It then passes this concatenated tensor through a `merge_time` layer (presumably a custom layer defined elsewhere in the code), which reduces the dimensionality from `(bs, seq, time_dim * time_num)` to `(bs, seq, user_dim)`. This likely involves some form of aggregation or projection operation to transform the high-dimensional time embeddings into a lower-dimensional user embedding.
- 
- In summary, the code first combines multiple time embeddings into one large tensor and then reduces this tensor's dimensionality to produce a more compact user embedding that can be used for further processing in the model. This process helps in in

Testing:  95%|█████████▌| 20/21 [05:45<00:18, 18.80s/it]

Generated Answer for question 'Answer:':
- The code snippet provided appears to be part of a function that processes time embeddings in a sequential recommendation model. Specifically, it concatenates multiple time embeddings along the last dimension and then applies a merge operation to transform them into a user embedding of a different dimensionality. This process is likely used to capture temporal dynamics in sequences of user interactions, which is crucial for predicting the next item in a sequence.
- 
- The context explains that this model predicts the next item in a sequence of user interactions, unlike traditional language models that operate on text inputs and outputs. The method described here aims to handle sequential data more efficiently by processing a sequence of previous interactions and generating an embedding for the next item. The use of time embeddings suggests that the model takes into account the timing of interactions, which can be important for understanding use

Testing: 100%|██████████| 21/21 [05:49<00:00, 16.66s/it]

Generated Answer for question 'Answer:':
- What does the User LLM output corresponding to Ei represent?
- The User LLM output corresponding to Ei represents E′i+1, which is expected to be the embedding of the next interaction Ii+1 in the sequence of previous interactions. In other words, it predicts the embedding of the next item in the sequence based on the current item Ei. This approach differs from traditional LLMs that process text inputs and outputs, as it focuses on predicting the next item in a sequence of interactions.

Question 1: Match? False
Answer: - The code snippet provided describes a process for generating an embedding for the next item in a sequence of user interactions. Specifically, it involves concatenating time embeddings across different time steps and then merging them into a single embedding that represents the user's interaction history up to the current point. This embedding is intended to capture the temporal dynamics of the interactions, which can help impro